Hill Climbing
=======

# Theory

## Definition


## Examples


# Implementation


## Imports

In [8]:
import numpy as np
import sympy as sp
from sympy.abc import x
from typing import Callable
import matplotlib.pyplot as plt
import warnings

## Parameters

In [9]:
x_0 = 2.5           # method starting value
generate_neighborhood = lambda x : [x + h for h in []]          # step size
func = x**2         # use sp. methods only "i.e sp.cos()"

## Algorithm


In [10]:
def HC(cost_func: Callable, generate_neighborhood: Callable, x_0: float, maxiter: int = int(5e2), task: str = "minimization") -> tuple[float, list, int]:


    x = np.zeros(maxiter)
    x[0] = x_0

    for i in range(maxiter):
        
        neighbors = generate_neighborhood(x[i])
        best_neighbor = neighbors[ cost_func(neighbors).argsort()[0] ] 

        # neighbor is better
        if cost_func(best_neighbor) < cost_func(x[i]):
            x[i+1] = best_neighbor

        if cost_func(best_neighbor) > cost_func(x[i]):
            break
        
    
    x_min, history, steps= x[i+1], x[:i+2], i+1

    return x_min, history, steps

## Results Visualization

In [11]:
# domain
xx = np.linspace(*domain)

# function
f = sp.lambdify('x', func)

# function derivative
f_prime = sp.lambdify('x', sp.diff(func))

# gradient decent
x_min, history, steps = HC(cost_func=func, generate_neighborhood=generate_neighborhood, x_0=x_0)

TypeError: 'Pow' object is not callable